In [1]:
import pandas as pd
import numpy as np

In [2]:
pick_num2str = dict()

for pick in range(1, (12*15)+1):
    num = pick%12
    rounds = pick//12 + 1
    
    if num == 0:
        num = 12
        rounds -= 1
        
    pick_str = "%d.%02d"%(rounds, num)
    pick_num2str[pick] = pick_str
    
pick_str2num = {v: k for k, v in pick_num2str.items()}

In [3]:
class player():
    
    def __init__(self, name, order):
        self.name = name
        self.order = order
        
        odd_picks = list(range(order, (12*15)+1, 12))
        even_picks = list(range(13-order, (12*15)+1, 12))
        self.picks = odd_picks[0::2] + even_picks[1::2]
        self.picks.sort()
        
        self.orig_picks = self.picks.copy()
    
    def trade(self, send, to, receive, how = "picks"):
        if how == "rounds":
            send_pick_nums = []
            for r in send:
                send_pick_nums.append(self.orig_picks[r-1])
            
            receive_pick_nums = []
            for r in receive:
                receive_pick_nums.append(to.orig_picks[r-1])
        else:
            send_pick_nums = [pick_str2num[s] for s in send]
            receive_pick_nums = [pick_str2num[s] for s in receive]
        
        self.send_picks(send_pick_nums)
        self.receive_picks(receive_pick_nums)

        to.receive_picks(send_pick_nums)
        to.send_picks(receive_pick_nums)

    def send_picks(self, picks):
            for n in picks:
                self.picks.remove(n)
            self.picks.sort()

    def receive_picks(self, picks):
        for n in picks:
            self.picks.append(n)            
        self.picks.sort()
    
    def str_picks(self):
        return [pick_num2str[n] for n in self.picks]
    
    def str_orig_picks(self):
        return [pick_num2str[n] for n in self.orig_picks]
        

In [4]:
cheese = player("Cheese", 1)
fritz = player("Fritz", 2)
topher = player("Topher", 3)
spencer = player("Spencer", 4)
jonathan = player("Jonathan", 5)
werf = player("Werf", 6)
lanu = player("Lanu", 7)
jeremy = player("Jeremy", 8)
micky = player("Micky", 9)
hair = player("Hair", 10)
yiding = player("Yiding", 11)
matt = player("Matt", 12)

In [5]:
# Pre draft order determination trades

yiding.trade([1], werf, [15], how = "rounds")
yiding.trade([3], spencer, [9], how = "rounds")

matt.trade([1], lanu, [7], how = "rounds")

jonathan.trade([2], lanu, [15], how = "rounds")

hair.trade([1], cheese, [7], how = "rounds")
hair.trade([3, 9], jeremy, [14, 15], how = "rounds")

topher.trade([3,10], fritz, [6,15], how = "rounds")
topher.trade([2], jeremy, [9], how = "rounds")

micky.trade([13], yiding, [15], how = "rounds")

In [6]:
# Post draft order determination trades
jonathan.trade(["1.05", "3.05", "8.08"], lanu, ["1.12", "2.06", "9.07"])

jeremy.trade(["6.05", "9.10"], jonathan, ["6.08", "9.05"])
jeremy.trade(["8.05", "9.05"], cheese, ["8.12", "9.01"])

werf.trade(["6.07", "7.06"], yiding, ["5.11", "9.11"])

In [7]:
draft_order_dict = dict()

players = [cheese, fritz, topher, spencer, jonathan, werf, lanu, jeremy, micky, hair, yiding, matt]

for play in players:
    for n in play.picks:
        draft_order_dict[n] = play.name
    

In [8]:
pick_order = pd.DataFrame(list(range(1,12*15+1)), columns = ["Pick Number"])
pick_order["Team"] = pick_order["Pick Number"].map(draft_order_dict)
pick_order

,Pick Number,Team
0,1,Cheese
1,2,Fritz
2,3,Topher
3,4,Spencer
4,5,Lanu
5,6,Werf
6,7,Lanu
7,8,Jeremy
8,9,Micky
9,10,Cheese


In [9]:
draft_board = np.arange(12*15).reshape(15,12)+1

for i in range(1,15,2):
    draft_board[i] = draft_board[i][::-1]

draft_board = pd.DataFrame(draft_board)
draft_board = draft_board.replace(draft_order_dict)
draft_board

,0,1,2,3,4,5,6,7,8,9,10,11
0,Cheese,Fritz,Topher,Spencer,Lanu,Werf,Lanu,Jeremy,Micky,Cheese,Werf,Jonathan
1,Cheese,Fritz,Jeremy,Spencer,Lanu,Werf,Jonathan,Jeremy,Micky,Hair,Yiding,Matt
2,Cheese,Fritz,Fritz,Spencer,Lanu,Werf,Lanu,Jeremy,Micky,Jeremy,Spencer,Matt
3,Cheese,Fritz,Topher,Spencer,Jonathan,Werf,Lanu,Jeremy,Micky,Hair,Yiding,Matt
4,Cheese,Fritz,Topher,Spencer,Jonathan,Werf,Lanu,Jeremy,Micky,Hair,Werf,Matt
5,Cheese,Topher,Topher,Spencer,Jeremy,Yiding,Lanu,Jonathan,Micky,Hair,Yiding,Matt
6,Hair,Fritz,Topher,Spencer,Jonathan,Yiding,Matt,Jeremy,Micky,Hair,Yiding,Matt
7,Jeremy,Fritz,Topher,Spencer,Lanu,Werf,Lanu,Cheese,Micky,Hair,Yiding,Matt
8,Jeremy,Fritz,Topher,Yiding,Cheese,Werf,Jonathan,Topher,Micky,Jonathan,Werf,Matt
9,Cheese,Fritz,Fritz,Spencer,Jonathan,Werf,Lanu,Jeremy,Micky,Hair,Yiding,Matt
